In [ ]:
from roboflow import Roboflow
import os
from  ultralytics import YOLO

In [4]:
import yaml
dataset = "./D-Fire/"
data = {
    'names': ['fire', 'smoke'],
    'nc': 2,
    'val': 'test/images',
    'train': 'train/images'
}

# YAML 파일로 저장
with open(f'{dataset}data.yaml', 'w') as file:
    yaml.dump(data, file)

print("YAML 파일이 생성되었습니다.")



YAML 파일이 생성되었습니다.


In [1]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 2 to use
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from ultralytics import YOLO
import cv2
import timeit
dataset = "C:/Users/LGS/Desktop/STEP_7/fire_det/D-Fire/"
# predict on a local image
model = YOLO("yolov8x.pt")
model.train(data=f"{dataset}/data.yaml", 
            epochs=50,
           batch = 32) #s -> 64 -> 15.8g

New https://pypi.org/project/ultralytics/8.0.121 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.119  Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=C:/Users/LGS/Desktop/STEP_7/fire_det/D-Fire//data.yaml, epochs=50, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=Fals

val: Scanning C:\Users\LGS\Desktop\STEP_7\fire_det\D-Fire\test\labels.cache... 4306 images, 2005 backgrounds, 8 corrupt: 100%|██████████| 4306/4306 [00:00<?, ?it/s]
val: WARNING  C:\Users\LGS\Desktop\STEP_7\fire_det\D-Fire\test\images\WEB10769.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0297]
val: WARNING  C:\Users\LGS\Desktop\STEP_7\fire_det\D-Fire\test\images\WEB10770.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0078]
val: WARNING  C:\Users\LGS\Desktop\STEP_7\fire_det\D-Fire\test\images\WEB10775.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0156]
val: WARNING  C:\Users\LGS\Desktop\STEP_7\fire_det\D-Fire\test\images\WEB10821.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0094]
val: WARNING  C:\Users\LGS\Desktop\STEP_7\fire_det\D-Fire\test\images\WEB11090.jpg: ignoring corrupt image/label: non-normalized or out of bounds c


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      12/50      24.9G      1.407      1.218      1.433         10        640: 100%|██████████| 539/539 [07:56<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:45<00:00,  1.51it/s]
                   all       4298       5176      0.705      0.651      0.711      0.395

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      13/50      24.8G       1.39      1.184      1.423          7        640: 100%|██████████| 539/539 [07:57<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:45<00:00,  1.51it/s]
                   all       4298       5176      0.693      0.655      0.707      0.391

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      14/50      24.9G      1.379       1.15      1

In [1]:
from ultralytics import YOLO
import cv2
import timeit

model = YOLO('C:/Users/LGS/Desktop/STEP_7/fire_det/runs/detect/train23/weights/best.pt')
video_name = "test_fire"
video_path = f"C:/Users/LGS/Desktop/STEP_7/fire_det/test_mp4/{video_name}.mp4"
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(f'{video_name}_result.mp4', fourcc, cap.get(5), (int(cap.get(3)),int(cap.get(4))))
# out2 = cv2.VideoWriter('pose.mp4', fourcc, cap.get(5), (int(cap.get(3)),int(cap.get(4))))


# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        start_time = timeit.default_timer()

        # Run YOLOv8 inference on the frame
        results = model.predict(frame,verbose = False, conf = 0.7)[0]
        
        # Visualize the results on the frame
        tracking = results.plot(prob = False, conf = False)
        terminate_time = timeit.default_timer()
        fps = str(int(1./(terminate_time - start_time )))
        # cv2.rectangle(tracking,(0,0), (150,50), (255,255,255), -1)
        # cv2.putText(tracking, "fps = {} ".format(fps), (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0))
        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", tracking)
        
        out.write(tracking)
        # out2.write(pose_tracking)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
out.release()
# out2.release()

cv2.destroyAllWindows()

In [29]:
import json
json_data = json.loads(results.tojson())

In [34]:
from datetime import datetime

for label in json_data:
    print(datetime.now(),label['name'])

2023-06-26 14:14:17.749888 smoke
2023-06-26 14:14:17.749888 fire
